### Adaptive Derivative-Assembled Pseudo-Trotter ansatz Variational Quantum Eigensolver [1] (ADAPT-VQE)

The Unitary Coupled Clusters truncated at the level of double excitations (UCCSD)
$$ |\psi^{UCCSD}\rangle \equiv e^{T(1)+T(2)}|\psi^{HF}\rangle, \label{eq:1}\tag{1}$$
where
$$ T(1)=\sum_{ia} \hat \tau_{ia} = \sum_{ia} \tau_{ia} (a^{\dagger}_{a} a_{i} - a^{\dagger}_{i} a_{a}),$$
$$T(2) = \sum_{iajb} \hat \tau_{iajb} = \sum_{iajb} \tau_{iajb} (a^{\dagger}_{a}  a^{\dagger}_{b} a_{j} a_{i} - a^{\dagger}_{i} a^{\dagger}_{j} a_{b} a_{a}).$$
is a natural ansatz for Variational Quantum Eigensolver since it is parametrized by the unitary operators and has a polynomial number of variational parameters. For the practical application on the quantum computer, it is desired to decompose unitary operator in Eq. (\ref{eq:1}) into a product of unitary operators acting on a few qubits. This can be achieved via the first-order Trotter decomposition. Because of the variational flexibility of the ansatz (parameters are optimized after the Trotterization), one can sufficiently reproduce the UCCSD results performing only single Trotter step
$$ |\psi^{UCCSD}\rangle \approx \prod_{ia}e^{\hat \tau_{ia}} \prod_{iajb}e^{\hat \tau_{iajb}} |\psi^{HF}\rangle. \label{eq:2}\tag{2}$$
The ADAPT-VQE algorithm suggests limiting the number of unitary operators in Eq. (\ref{eq:2}) to lower the depth of the circuit and make the ansatz even more practical for quantum computing. This is achieved by the systematic extension of the ansatz by one excitation operator at a time, starting from the Hartree Fock state. Specifically, having specified the pool of all possible single and double excitation operators
$\{ \hat A_1, \hat A_2, \dots, \hat A_n \}$ ($\hat A_i \in \{ a^{\dagger}_a a_i - a^{\dagger}_i a_a; a^{\dagger}_{a}  a^{\dagger}_{b} a_{j} a_{i} - a^{\dagger}_{i} a^{\dagger}_{j} a_{b} a_{a} \}$), at first one evaluates the gradient $$\frac{\partial E}{\partial \theta_m} = 
\left.\frac{\partial}{\partial \theta_m} \langle \psi^{HF} |e^{-\theta_1 \hat A_1} e^{-\theta_2 \hat A_2} \ldots
 \hat H \ldots e^{\theta_2 \hat A_2} e^{\theta_1 \hat A_1}| \psi^{HF} \rangle \right|_{ \forall \theta_i=0} 
 =\langle \psi^{HF} | [ \hat H,\hat A_m] | \psi^{HF} \rangle . \label{eq:3}\tag{3}$$
 Then ansatz is extended with the excitation operator,$\hat A_k$, corresponding to the highest element of the gradient. Afterwards, all or only the new variational parameters are optimized. Continuing this process until the energy, the L$^2$ norm of the gradient or the maximum element is converged one recovers ADAPT-VQE [1]
$$|\psi^{ADAPT}\rangle \equiv (e^{\hat \tau_N}) (e^{\hat \tau_{N-1}}) \ldots (e^{\hat \tau_2}) (e^{\hat \tau_1}) |\psi^{HF}\rangle, \label{eq:4}\tag{4}$$
where $\hat \tau_p$ can be either $\hat \tau_{ia}$ or $\hat \tau_{iajb}$. It should be emphasized that the operator  pool is never drained and always contains all possible single and double excitation operators, thus allowing some excitation operator,$\hat A_k$, enter the ansatz multiple times.

[1] Harper R. Grimsley, Sophia E. Economou, Edwin Barnes & Nicholas J. Mayhall, *Natur. Commun.* 10, 3007 (2019)

In [7]:
import numpy as np

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.formats import MoleculeInfo
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit_nature.second_q.transformers import FreezeCoreTransformer

from qiskit_algorithms.optimizers import SLSQP
from qiskit_algorithms.minimum_eigensolvers import AdaptVQE, VQE

from qiskit.primitives import Estimator

### Molecule Setup

The lithium hydride  at the equilibrium bond length (~1.5 Angstrom) is studied in this tutorial. In order to set up the molecular Hamiltonian one has to get the one- and two-electron integrals from a computational chemistry driver (PySCF in this case) employing the Hartree Fock calculation in STO-3G basis set. The information from the driver is saved in the ``molecule`` object. Then the Fermionic Hamiltonian is mapped to the qubit Hamiltonian using the ``parity`` mapping type as it further reduces the problem size.

In [45]:
# Use PySCF, a classical computational chemistry software
# package, to compute the one-body and two-body integrals in
# molecular-orbital basis, necessary to form the Fermionic operator

info = MoleculeInfo(["Li", "H"], [(0.0, 0.0, 0.0), (0.0, 0.0, 1.5)])

driver = PySCFDriver.from_molecule(info, basis="sto3g")
molecule = driver.run()

transformer = FreezeCoreTransformer()
molecule = transformer.transform(molecule)
hamiltonian = molecule.hamiltonian.second_q_op()
mapper = ParityMapper(num_particles=molecule.num_particles)
tapered_mapper = molecule.get_tapered_mapper(mapper)
qubit_op = tapered_mapper.map(hamiltonian)

### Initial State and Optimizer
Set the initial state to the Hartree Fock state, $| \psi^{HF} \rangle$, and choose the optimizer for the VQE algorithm.

In [46]:
# setup the initial state for the variational form
init_state = HartreeFock(
            molecule.num_spatial_orbitals,
            molecule.num_particles,
            tapered_mapper,
        )

estimator = Estimator()

optimizer = SLSQP(maxiter=10000, ftol=1e-9)

### UCCSD-VQE reference
Perform the reference calculation employing the ``UCCSD`` variational form and ``VQE`` algorithm.

In [47]:
vqe_ansatz = UCCSD(
    molecule.num_spatial_orbitals,
    molecule.num_particles,
    tapered_mapper,
    initial_state=init_state
)
vqe = VQE(estimator, vqe_ansatz, optimizer)
vqe.initial_point = [0] * vqe_ansatz.num_parameters
algo = GroundStateEigensolver(tapered_mapper, vqe)
result_vqe = algo.solve(molecule)
energy_vqe = result_vqe.eigenvalues[0]

### ADAPT-VQE run
Now re-instantiate a new ``UCCSD`` object for the use with the ``AdaptVQE`` algorithm (https://qiskit-community.github.io/qiskit-algorithms/stubs/qiskit_algorithms.AdaptVQE.html)

In [48]:
adapt_vqe_ansatz = UCCSD(
    molecule.num_spatial_orbitals,
    molecule.num_particles,
    tapered_mapper,
    initial_state=init_state
)

adapt_vqe = AdaptVQE(VQE(estimator, adapt_vqe_ansatz, optimizer))
result_adapt_vqe = adapt_vqe.compute_minimum_eigenvalue(qubit_op)
energy_adapt_vqe = result_adapt_vqe.eigenvalue

### Compare results
Print and compare the gates utilized in both parameterizations and the corresponding energies.

In [50]:
from prettytable import PrettyTable

table = PrettyTable()
table.field_names = ["Ansatz","Energy (Hartree)","Gates"]

vqe_cirq = result_vqe.raw_result.optimal_circuit
table.add_row(['UCCSD', str(energy_vqe), vqe_cirq.count_ops()])

adapt_vqe_cirq = result_adapt_vqe.optimal_circuit
table.add_row(['ADAPT-VQE', str(result_adapt_vqe.eigenvalue), adapt_vqe_cirq.count_ops()])

print(table)

+-----------+---------------------+----------------------------------+
|   Ansatz  |   Energy (Hartree)  |              Gates               |
+-----------+---------------------+----------------------------------+
|   UCCSD   | -1.1001883328961515 | OrderedDict([('EvolvedOps', 1)]) |
| ADAPT-VQE | -1.1001795367631413 | OrderedDict([('EvolvedOps', 1)]) |
+-----------+---------------------+----------------------------------+


In [44]:
! pip freeze | grep qiskit

qiskit==1.1.0
qiskit-aer==0.14.2
qiskit-algorithms==0.3.0
qiskit-ibm-runtime==0.25.0
qiskit-machine-learning==0.7.2
qiskit-nature==0.7.2
qiskit-nature-pyscf==0.4.0
qiskit-qasm3-import==0.5.0
qiskit-transpiler-service==0.4.5
